# Suivis Projet Info : Stellar Analytics
#### Axelle Refeyton . Jean Roelens

Le but de se projet est initialement de permettre à un utilisateur de prendre une photo d'une étoile et de retrouver des informations dessus, notamment sont nom, sa position ainsi que sa composition chimique qui peux être récupérer en fonction de son spectre lumineux.

Ressources utilisées : 
- [NIST Spectra Database](https://physics.nist.gov/PhysRefData/ASD/lines_form.html)
- [Documentation sur les spectres lumineux](https://atomic-spectra.net/)
- [Simbad, DB d'étoiles communautaire open source](https://simbad.u-strasbg.fr/simbad/)
- [MAST portal](https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html)
- [Raies de Fraunhofer](https://fr.wikipedia.org/wiki/Raies_de_Fraunhofer)

En début de projet beaucoup de temps fut consacré à de la recherche de documentation ainsi que de jeux de données. Egalement nous avons eu l'occasion de nous entretenir lors d'une conférence Zoom avec deux membres du CNES (Orphée Faucoz et Denis Standarovski) travaillant depuis plusieurs années sur un sujet similaire : L'identification d'exoplanétes en utilisant les spectres d'absorption.

Au cours de l'entretien ils nous ont clairement expliqué que ce que l'on souhaiter faire êtait une tâche difficile et que cela nécessité du temps ne serait-ce que pour la compréhension de l'environnement dans lequel nous naviguons.

On a parcouru beaucoup de base de données gratuite et d'autres non afin de trouver celles qui correspondrait au mieux à nos besoins. Au final on a retenue Simbad pour la récupération des spectres émis par les étoiles et NIST pour récupérer le spectre d'émissions des élèments que peuvent composer une étoile, càd : Oxygéne, Hydrogénes, Sodium, Hélium, Mercure, Fer, Magnésium, Calcium, Titane et finalement Nickel.

à partir de ce moment on a divisé la structure du projet en deux parties, une partie objet céleste (gérer par Axelle Refeyton) et une partie spectre chimique (gérer par Jean Roelens). Ensuite chaque partie est divisés en étapes afin de faciliter le procéder de réalisation.

## Objet Céleste (A.Refeyton)

## Spectre Chimique (J.Roelens)

L'objectif de cette partie est :
1. **Pour chaque élèments récupérer les différentes ionization possible.**
2. **Extraire le tableau de spectre depuis NIST (en utilisant un scrapper)**
3. **Le formatter pour récupérer que les lignes fortes**
4. **Créer un dataset par élément et ionization avec les lignes fortes**

Pour commencer il faut que je récupére les ionizations de chaque élèments, étant une étape pas forcément très complexe j'ai préféré le faire à la main en m'aidant de [atomic-spectra](https://atomic-spectra.net/spectrum.php) et des [Raies de Fraunhofer](https://fr.wikipedia.org/wiki/Raies_de_Fraunhofer) On en sort cette liste : 
- Oxygéne [I, II, III, IV, V, VI, VII, VIII]
- Hydrogénes [I]
- Sodium [I, II, III, IV, V, VI, VII, VIII, IX, X, XI]
- Hélium [I, II]
- Mercure [I, II, III]
- Fer [I, II ,III , IV, V, VI, VII, VIII, IX, X, XI, XII, XIII, XIV, XV, XVI, XVII, XVIII, XIX, XX, XXI, XXII, XXIII, XXIV, XXV, XVI]
- Magnésium [I, II, III, IV, V, VI, VII, VIII, IX, X, XI, XII]
- Calcium [I, II, III, IV, V, VI, VII, VIII, IX, X, XI, XII, XIII, XIV, XV, XVI]
- Titane [I, II, III, IV, V, VI, VII, VIII, IX, X, XI, XII, XIII, XIV, XV, XVI, XVII, XVIII, XIX, XX, XXI, XXII]
- Nickel [I, II, III, IV, V, VI, VII, VIII, IX, X, XI, XII, XIII, XIV, XV, XVI, XVII, XVIII, XIX, XX, XXI, XXII, XXIII, XXIV, XXV, XXVI, XXVII, XXVIII]

Maintenant que l'étape facile est faites je vais pouvoir m'attaquer à l'étape plus hardu d'extraire les données depuis NIST. L'idée global dérriére est de récupérer le HTML de la page NIST en donnant dans l'URL du site l'élèment ainsi que son indice de ionization.

En saisissant H I (pour Hydrogénes Ionization 1) on obtiens cet URL :

https://physics.nist.gov/cgi-bin/ASD/lines1.pl?spectra=H+I&output_type=0&low_w=&upp_w=&unit=1&submit=Retrieve+Data&de=0&plot_out=0&I_scale_type=1&format=0&line_out=0&en_unit=0&output=0&bibrefs=1&page_size=15&show_obs_wl=1&show_calc_wl=1&unc_out=1&order_out=0&max_low_enrg=&show_av=2&max_upp_enrg=&tsb_value=0&min_str=&A_out=0&intens_out=on&max_str=&allowed_out=1&forbid_out=1&min_accur=&min_intens=&conf_out=on&term_out=on&enrg_out=on&J_out=on

Plutôt long, mais ce qui est assez intéressant que beaucoup de paramètres pourront-être passé assez facilement par celles-ci comparons avec l'URL de O I pour voir ou nous devrons saisir l'élèment et l'ionization 

https://physics.nist.gov/cgi-bin/ASD/lines1.pl?spectra=O+I&output_type=0&low_w=&upp_w=&unit=1&submit=Retrieve+Data&de=0&plot_out=0&I_scale_type=1&format=0&line_out=0&en_unit=0&output=0&bibrefs=1&page_size=15&show_obs_wl=1&show_calc_wl=1&unc_out=1&order_out=0&max_low_enrg=&show_av=2&max_upp_enrg=&tsb_value=0&min_str=&A_out=0&intens_out=on&max_str=&allowed_out=1&forbid_out=1&min_accur=&min_intens=&conf_out=on&term_out=on&enrg_out=on&J_out=on

Comme on peut le voir l'élèment se trouve en début d'URL essayons cela.

In [69]:
import requests

START_URL = r"https://physics.nist.gov/cgi-bin/ASD/lines1.pl?spectra="
END_URL = r"&output_type=0&low_w=&upp_w=&unit=1&submit=Retrieve+Data&de=0&plot_out=0&I_scale_type=1&format=0&line_out=0&en_unit=0&output=0&bibrefs=1&page_size=15&show_obs_wl=1&show_calc_wl=1&unc_out=1&order_out=0&max_low_enrg=&show_av=2&max_upp_enrg=&tsb_value=0&min_str=&A_out=0&intens_out=on&max_str=&allowed_out=1&forbid_out=1&min_accur=&min_intens=&conf_out=on&term_out=on&enrg_out=on&J_out=on"

# Essayons de voir si on peut obtenir un code 200, mais avant cela construisons notre URL de maniére naïve pour ce test

URL = START_URL + "Ni+X" + END_URL

resp = requests.get(URL)
if resp.status_code == 200:
    print(f"Succés de la requéte")
else :
    print(f"Echec de la requéte")

Succés de la requéte


Maintenant que notre approche naïve fonctionne vérifions que l'intégralité des élèments et ionization fonctionne.

In [70]:
# Le fait que je marque certaines "variables" en majuscules est simplement un tic pour m'indiquer qu'il s'agit de constante
ELEMENT_DICT = {
    "O" : ["I", "II", "III", "IV", "V", "VI","VII"],
    "H" : ["I"],
    "Na" : ["I", "II", "III", "IV", "V", "VI","VII","VIII","IX","X","XI"],
    "He" : ["I","II"],
    "Hg" : ["I","II","III"],
    "Fe" : ["I", "II", "III", "IV", "V", "VI","VII","VIII","IX","X","XI","XII","XIII","XIV","XV","XVI"],
    "Mg" : ["I", "II", "III", "IV", "V", "VI","VII","VIII","IX","X","XI","XII"],
    "Ca" : ["I", "II", "III", "IV", "V", "VI","VII","VIII","IX","X","XI","XII","XIII","XIV","XV","XVI"],
    "Ti" : ["I", "II", "III", "IV", "V", "VI","VII","VIII","IX","X","XI","XII","XIII","XIV","XV","XVI","XVII","XVIII","XIX","XX","XXI","XXII"],
    "Ni" : ["I", "II", "III", "IV", "V", "VI","VII","VIII","IX","X","XI","XII","XIII","XIV","XV","XVI","XVII","XVIII","XIX","XX","XXI","XXII","XXIII","XXIV","XXV","XXVI","XXVII","XXVIII"]
}

# Je vérifie la longueur de chacune des clès pour être sur de ne pas avoir fait d'erreur 
def element_dict_verification():
    for key in ELEMENT_DICT.keys():
        print(f"Elément : {key} Len : {len(ELEMENT_DICT[key])}")

# Et ensuite on test les requêtes

def get_200(r : requests.models.Response):
    if r.status_code != 200:
        return False
    return True
def multi_request_verification(d : dict):
    for k in d.keys():
        for v in range(len(d[k])):
            url = START_URL + k + "+" + d[k][v] + END_URL
            if not get_200(requests.get(url)):
                print(f"Error with el : {k} and ionization : {d[k][v]}")
            else : 
                print(f"Success with el : {k} and ionization : {d[k][v]}")
# multi_request_verification(ELEMENT_DICT)

Ok on dirait bien qu'on a que des codes 200 pour l'ensemble des requêtes ! Malheuresement on se rendra compte plus tard que NIST renvoie des code 200 même en cas de mauvais URL. Le seul moyen de savoir si la requête a bien aboutit et de récupérer le tableau **"\<table>"** qui va contenir les données qui nous intéresse. 

L'idée va être de voir si on peut récupérer la balise "table" contenant le styling que nous cherchons pour chaque lien avec l'aide de la librairie que nous allons le plus utiliser durant notre partie : "BeautifulSoup"

In [71]:
from bs4 import BeautifulSoup
from pprint import pprint

# Vu que je vais beaucoup utiliser request autant se créer des petites fonctions QOL (Quality Of Life)

def url_builder(d : dict):
    resp = {}
    for key in d.keys():
        for v in d[key]:
            resp[key+v] = START_URL + key + "+" + v + END_URL
    return resp

In [72]:
urls = url_builder(ELEMENT_DICT)
for url in urls:
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')
    tables = soup.find_all('table')
    target_table = [table for table in tables if 'background-color:#FFFEEE;' in table.get('style', '')]
    if len(target_table) != 1:
        print(f'error with url : {url}')
    else :
        continue

MissingSchema: Invalid URL 'OI': No scheme supplied. Perhaps you meant https://OI?

On voit donc désormais que deux lien sont *mort* celui pour le Nickel 6 et le Nickel 8. Hormis ça l'ensemble des autres liens marche. Pour la prochaine étape il nous faut récupérer toute les rows contenant une donnée dans la colonne *Observed Wavelength Vac (nm)* ainsi que *Rel. Int.* C'est les deux colonnes qui nous intéresse pour la suite.

In [76]:
# Pour commencer passont la logique précédente en fonction 

def get_table(req : requests.models.Response):
    soup = BeautifulSoup(req.text,'html.parser')
    tables = soup.find_all('table')
    target_table = [table for table in tables if 'background-color:#FFFEEE;' in table.get('style', '')]
    if len(target_table) > 0 :
        return target_table[0]
    else :
        return None

# On va traiter seulement le premier question de rapidité depuis python 3.6 les dicts sont ordonnées du coup le premier élèment sera toujours OI
urls = url_builder(ELEMENT_DICT)
oxygen_one = list(urls.items())[0]
table = get_table(requests.get(oxygen_one[1]))
# Maintenant on va "drill down" la table afin d'obtenir tout les Tbody puis les Tr qui nous intéressent.
tbodies = table.find_all('tbody')
# Seul les tbody pair nous intéresse car ils contiennent la data en question
evenTBodies = tbodies[1::2]
# Ensuite pour chaque tbody on veut récupérer toute les balises tr
# firstTBody = evenTBodies[0]
# Holder pour les values récupéré afin d'en faire un dataframe par la suite
All_Observed_Wavelength = []
All_Relative_Intensities = []

# for tbody in evenTBodies :
#     for tr in tbody.find_all('tr'):
#         all_td = tr.find_all('td')
#         if len(all_td) >= 2:
#             Obs_Wav = all_td[0].text.strip()
#             Rel_Int = all_td[2].text.strip()
#             if Obs_Wav != "" and Rel_Int != "":
#                 All_Observed_Wavelength.append(Obs_Wav)
#                 All_Relative_Intensities.append(Rel_Int)
# pprint(All_Observed_Wavelength)

# Tiens on remarque que la structure du HTML doit avoir un petite erreur car lors du print de All_Observed_Wavelength 
# on remarque qu'on print les bonnes colonnes avec les bonnes valeur mais, on a 1 fois le tableau 1, 2 fois le tableau 2, et finalement 3 fois le tableau
# 3 ce qui nous indique que l'on peut parcourir une seul fois le tableau 1 et récupérer toutes les données.

tbody = tbodies[1]
for tr in tbody.find_all('tr'):
    all_td = tr.find_all('td')
    if len(all_td) >= 2:
        obs_wav = all_td[0].text.strip()
        rel_int = all_td[2].text.strip()
        if obs_wav != "" and rel_int != "":
            All_Observed_Wavelength.append(obs_wav)
            All_Relative_Intensities.append(rel_int)
import pandas as pd

df = pd.DataFrame({
    'wavelength' : All_Observed_Wavelength,
    'intensities': All_Relative_Intensities,
    'chemical' : 'O I'
})

print(df.describe())

       wavelength intensities chemical
count         174         174      174
unique        157          48        1
top       94.8686         120      O I
freq            3          18      174


## Jonction entre spectre inconnu et ce qui le compose

Ici le but va être d'utiliser les datasets réaliser dans la partie "Spectre Chimique" pour faire une proposition de la composition du spectre inconnu. Pour chaque paire élèment/ionization il faut fournir un indice de confiance de la présence du dis élèment.